In [ ]:
import requests
import pandas as pd
import json
from datetime import datetime, date, timedelta
import numpy as np
import os
import urllib3
urllib3.disable_warnings()

from openpyxl.styles import PatternFill
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl import Workbook

In [ ]:
token = "PnWxSncrbEolYFc"
url = 'https://rosakhutor.foquz.ru/foquz/api/v1/poll/list'
query = {'access-token': token, 'page': 1, 'perPage': 1000}
all_data = []
while True:
            response = requests.get(url, params=query, verify=False)
            if response.status_code == 200:
                data = response.json()['polls']

                if not data:
                    break
                all_data.extend(data)
                query["page"] += 1              
all_polls = pd.json_normalize(all_data)

In [ ]:
# Прописать название опроса из фокуза

survey_name = 'Название опроса'
survey_id = all_polls.loc[all_polls['name'] == survey_name, 'id'].reset_index()['id'][0]

In [ ]:
url_2 = 'https://foquz.ru/foquz/api/v2/answers'
polls = [survey_id]
query = {'access-token': token, 'polls[]': polls, 'offset': 0, 'limit': 1}
poll_answers = []
while True:
            response = requests.get(url_2, params=query, verify=False)
            if response.status_code == 200:
                data = response.json()['items']
                if not data:
                    break
                poll_answers.extend(data)
                query["offset"] += 1              
survey = pd.json_normalize(poll_answers)

In [ ]:
survey_new = survey.rename(columns = {'points.max': 'poll_max', 'points.value':'poll_fact'})
survey_filt = survey_new.loc[survey_new['poll_max'] != 0]

In [ ]:
# Проверка на начатые, но незаконченные анкеты. Если None - значит открыли ссылку, но не заполнили даже первый вопрос.
indexes = survey_new.loc[survey_new['poll_max'] == 0].index
for i in indexes:
    print(f"{i} : {survey_new.loc[i].answers[0].get('answer', 'нет строки')}")
    
# Если есть не None, то надо разбираться, почему не дошли до конца. Сразу подозрение на разведку вопросов

In [ ]:
survey_filt_2 = survey_filt[['id', 'answer_date', 'sent_date', 'status', 'answers', 'poll_fact', 'poll_max']]
exploded_df = survey_filt_2.explode('answers', ignore_index=True)
m_df = pd.concat([exploded_df.drop('answers', axis=1), pd.json_normalize(exploded_df['answers'])], axis=1)
df_to_open = m_df.loc[m_df['answer'].isna() == False]

In [ ]:
def exploded_answers(df):
    new_rows = []
    for index, row in df.iterrows():
        if row['type'] == 'Варианты на выбор':
            new_row = row.to_dict()
            new_row['answer'] = '; '.join(item['title'] for item in row['answer'])
            new_rows.append(new_row)
        else:
            new_rows.append(row)
    new_df = pd.DataFrame(new_rows)
    return new_df

df_open = exploded_answers(df_to_open)

In [ ]:
df_open = df_open.rename(columns={'Наставник': 'tutor'})
df_open['student'] = ''
df_open['unit'] = ''

In [ ]:
def update_column(frame, window_column, target_column, key_column, question_value, answer_column):
    def update_value(x, values):
        for value in values:
            try:
                return x.loc[frame[key_column].str.contains(value)].iloc[0]
            except IndexError:
                pass
        return frame.loc[x.index[0], target_column]  
    frame[target_column] = frame.\
                        groupby([window_column])[answer_column].\
                        transform(lambda x: update_value(x, question_value))
    return frame

In [ ]:
df_to_analys = update_column(df_open, "id", "tutor", "question_text", ['Укажите ФИО наставника'], "answer")
df_to_analys = update_column(df_to_analys, "id", "student", "question_text", ['Укажите свое ФИО'], "answer")
df_to_analys = update_column(df_to_analys, "id", "unit", "question_text", ['Укажите Вашу службу'], "answer")

In [ ]:
df_to_analys = df_to_analys.rename(columns = {'points.max': 'answer_max', 'points.value':'answer_fact', 'id':'poll_id'})

In [ ]:
# Проверка на наличие более 1 заполненной анкеты у юзера

df_to_analys.loc[df_to_analys['question_text'] == 'Укажите свое ФИО']\
    .groupby('answer')\
    .agg(count = ('poll_id', 'count'), unique_id = ('poll_id', pd.Series.unique))\
    .query("count > 0")\
    .reset_index()\
    .rename(columns={'answer': 'ФИО', 'count':'количество', 'unique_id': 'id анкет'})

In [ ]:
# Исключение повторных ответов от 1 респондента после предварительного анализа
df_to_analys = df_to_analys.loc[~df_to_analys['poll_id'].isin(['84a7d915584b007873b370bd3a0bfed6',
                                                               '493c2324479d0ba13dc03e1d10ef7acb'])]

# Исключение прочего брака (руководительские, тестовые, разведывательские)
df_to_analys = df_to_analys.loc[~df_to_analys['poll_id'].isin(['0fb5406ceaaa46e822c04e29194cb15a',
                                                               '101a0694a0d5a7cbbb3e2270a0232653',
                                                               '8804896125a720baab10c1c01689a070'])]

In [ ]:
# Список id проверок, заполненных наставниками
id_tutors_polls = ['559e42571752441fa8a3a738a53de85a',
             '83fabd41943ffca651a50fb3f9b1d66f'
            ]
tutors_score = df_to_analys.loc[df_to_analys['poll_id'].isin(id_tutors_polls)]
student_score = df_to_analys.loc[~df_to_analys['poll_id'].isin(id_tutors_polls)].sort_values(['unit', 'tutor', 'student'])

In [ ]:
# Фильтрация информационных вопросов
student_ankets = student_score.loc[(student_score['question_text'] != 'Укажите Вашу службу')
                                  &(student_score['question_text'] != 'Укажите ФИО наставника')
                                  &(student_score['question_text'] != 'Укажите свое ФИО')]

In [ ]:
# Создание df для общих результатов (1 лист в отчете)
tutors = student_score.groupby('tutor', as_index = False).agg({'answer_fact': 'sum', 'answer_max': 'sum'})
tutors['Результат теста'] = round(tutors['answer_fact']/tutors['answer_max']*100, 2)
tutors = tutors.rename(columns = {'tutor': 'Наставник'})
tutors['Коэф. 1'] = 0.4
tutors['Результат аудиторских проверок (01.01.2025 - 31.03.2025)'] = [95.60, 93.73, 93.73, 90.90,
                                                                      93.56, 82.42, 96.13, 87.45,
                                                                      93.44, 87.45, 90.90]
tutors['Коэф. 2'] = 0.6
tutors['Формула расчета'] = 'результаты теста*0,4 (коэф. 1) + средний балл по аудиторским проверкам*0,6 (коэф. 2)'
tutors['Рейтинг за 1 квартал'] = round(tutors['Результат теста']*tutors['Коэф. 1'] + tutors['Результат аудиторских проверок (01.01.2025 - 31.03.2025)']*tutors['Коэф. 2'], 2)
tutors['Рейтинг за 4 квартал'] = [95.27, 91.64, 88.35, 80.14,
                                  90.91, 71.86, 81.39, 89.87,
                                  90.78, 90.31, 85.30]
tutors['Прорыв'] = round(tutors['Рейтинг за 1 квартал']-tutors['Рейтинг за 4 квартал'], 2)

In [ ]:
static_data = tutors[['Наставник', 'Результат теста', 'Коэф. 1',
       'Результат аудиторских проверок (01.01.2025 - 31.03.2025)', 'Коэф. 2',
       'Формула расчета', 'Рейтинг за 1 квартал', 'Рейтинг за 4 квартал',
       'Прорыв']].sort_values('Рейтинг за 1 квартал', ascending = False)

# Добавление тех, у кого подопечные не ответили/нет ретро данных
static_data.loc[len(static_data)] = ['Абрамченко Надежда Геннадьевна', '', 0.4,
                      89.18, 0.6,
                      'результаты теста*0,4 (коэф. 1) + средний балл по аудиторским проверкам*0,6 (коэф. 2)',
                      '', 85.95,'']

In [ ]:
unique_units = student_score['unit'].unique()
for unit_name in unique_units:
    df = student_ankets.loc[(student_score['unit'] == unit_name)]
    pivot_df = df.pivot_table(index=['poll_id', 'answer_date', 'unit', 'tutor', 'student', 'poll_fact'], 
                              columns='question_text', 
                              values='answer',
                              aggfunc='first').reset_index()

    points_dict = df.set_index(['poll_id', 'answer_date', 'unit',
                                'tutor', 'student', 'poll_fact', 'question_text'])['answer_fact'].to_dict()

    wb = Workbook()
    ws_static = wb.active
    ws_static.title = "Итог"
    for row in dataframe_to_rows(static_data, index=False, header=True):
        ws_static.append(row)

    pivot_df = pivot_df.rename(columns={
        'poll_id': 'ID анкеты',
        'answer_date': 'Дата',
        'unit': 'Служба',
        'tutor': 'Наставник',
        'student': 'Наставляемый',
        'poll_fact': 'Баллов набрано'
    }).sort_values(['Наставник', 'Баллов набрано'], ascending = [False, False])
    ws_results = wb.create_sheet("Результаты тестирования", 1)
    for r in dataframe_to_rows(pivot_df, index=False, header=True):
        ws_results.append(r)

    green_fill = PatternFill(start_color='90EE90', end_color='90EE90', fill_type='solid')
    red_fill = PatternFill(start_color='FF9999', end_color='FF9999', fill_type='solid')

    for col_idx, col in enumerate(ws_results.iter_cols(min_row=2, max_row=ws_results.max_row, min_col=7, max_col=ws_results.max_column), start=7):
        question = ws_results.cell(row=1, column=col_idx).value
        for row_idx, cell in enumerate(col, start=2):
            poll_fact   = ws_results.cell(row=row_idx, column=6).value
            student     = ws_results.cell(row=row_idx, column=5).value
            tutor       = ws_results.cell(row=row_idx, column=4).value
            unit        = ws_results.cell(row=row_idx, column=3).value
            answer_date = ws_results.cell(row=row_idx, column=2).value
            poll_id     = ws_results.cell(row=row_idx, column=1).value

            point = points_dict.get((poll_id, answer_date, unit, tutor, student, poll_fact, question), None)

            if point == 1:
                cell.fill = green_fill
            elif point == 0:
                cell.fill = red_fill

    # Прописать путь к папке на рабочем столе для сохранения файлов
    wb.save(f'C:/Users/papkov_ay/Desktop/Наставники_тесты/{unit_name}.xlsx')